In [7]:
from datetime import date

d0 = date(2008, 8, 18)
d1 = date(2008, 9, 26)
print type(d0)
delta = d1 - d0
print delta.days

<type 'datetime.date'>
39


In [12]:
import datetime
today = datetime.date.today()
someday = datetime.date(2008, 12, 25)
diff = someday - today
diff.days

-3172

In [1]:
'''
This program is to collect data for various IT positions with web scrapng from jobsdb website to analyse how many 
job openings for IT positions and what jobs are on high demand
'''
import math
import requests
import csv
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import datetime as dt
import re
from time import strptime

rcParams['figure.figsize']=(14,8)

url_base = "https://sg.jobsdb.com/sg/jobs/information-technology?KeyOpt=COMPLEX&JSRV=1&RLRSF=1&JobCat=131&JSSRC=JSRSB"
location='job_data.csv'
    
my_job_count_dict={
                    'Desktop Support':0,
                    'Business IT Analyst':0,
                    'Project Engineer':0,
                    'Application Support':0,
                    'IT Support':0,
                    'IT Administrator':0,
                    'Software Engineer':0,
                    'Network Engineer':0,
                    'Data Analyst':0,
                    'IT Consultant':0,
                    'SAP Engineer':0,
                    'Other IT Positions':0
                    }

page_num=0

#this function returns the total of IT jobs
def retrieve_total_number_of_job():
    url_to_scrape=url_base+"&page="+str(page_num)
    content =  requests.get(url_to_scrape).text
    soup = BeautifulSoup(content, 'html.parser')
    #retrieve_total_number_of_IT_jobs
    return int(soup.find("span",{"class":"criteria-primary"}).find("em").get_text())

number_of_job_tag=retrieve_total_number_of_job()
# print ("number_of_job_tag: {}".format(number_of_job_tag))

# get the total max number of pages of IT jobs
total_job_pages=int(math.ceil(number_of_job_tag/50.0))
# print ("total_job_pages: {}".format(total_job_pages))

# this function calculate the difference in days between today and last job posted
def calculate_diff_in_days(date_posted_tag): 
    #use regular expression class to split to day,month,year in list
    tlist=re.split("-",date_posted_tag["content"])
    
    # str(strptime(tlist[1],'%b').tm_mon) is to convert Month text to numeric
    # convert '30-Aug-17' to string 8/30/2017 format

    # convert to date object using datetime.date(year,month,day)
    someday = dt.date(int("20"+tlist[2]),strptime(tlist[1],'%b').tm_mon,int(tlist[0]))
    
    today = dt.date.today()
    #print today date object in mm/dd/yy format
#     print ("today is %s/%s/%s" % (today.month, today.day, today.year))
    
    return (today - someday).days

#sum up each IT post in each page
def sum_each_IT_post_count_in_page(job_title_tag):
    
    for each_job in job_title_tag:
        job_title=each_job.text.strip().lower()
        if "software engineer" in job_title:
            my_job_count_dict['Software Engineer']+=1
        elif "programmer" in job_title:
            my_job_count_dict['Software Engineer']+=1
        elif "developer" in job_title:
            my_job_count_dict['Software Engineer']+=1
        elif "development" in job_title:
            my_job_count_dict['Software Engineer']+=1
        elif "development" in job_title:
            my_job_count_dict['Software Engineer']+=1
        elif "sap" in job_title:
            my_job_count_dict['SAP Engineer']+=1
        elif "application support" in job_title:
            my_job_count_dict['Application Support']+=1
        elif "network" in job_title:
            my_job_count_dict['Network Engineer']+=1
        elif "security" in job_title:
            my_job_count_dict['Network Engineer']+=1
        elif "infrastructure" in job_title:
            my_job_count_dict['Network Engineer']+=1
        elif "project" in job_title:
            my_job_count_dict['Project Engineer']+=1
        elif "consultant" in job_title:
            my_job_count_dict['IT Consultant']+=1
        elif "business analyst" in job_title:
            my_job_count_dict['Business IT Analyst']+=1
        elif "business" in job_title:
            my_job_count_dict['Business IT Analyst']+=1
        elif "data" in job_title:
            my_job_count_dict['Data Analyst']+=1
        elif "dba" in job_title:
            my_job_count_dict['Data Analyst']+=1
        elif "administrator" in job_title:
            my_job_count_dict['IT Administrator']+=1
        elif "system" in job_title:
            my_job_count_dict['IT Administrator']+=1
        elif "desktop" in job_title:
            my_job_count_dict['Desktop Support']+=1
        elif "helpdesk" in job_title:
            my_job_count_dict['Desktop Support']+=1
        elif "desktop" in job_title:
            my_job_count_dict['Desktop Support']+=1
        elif "it support" in job_title:
            my_job_count_dict['IT Support']+=1
        elif "it executive" in job_title:
            my_job_count_dict['IT Support']+=1
        elif "it officer" in job_title:
            my_job_count_dict['IT Support']+=1
        elif "it executive" in job_title:
            my_job_count_dict['IT Support']+=1
        elif "it operation" in job_title:
            my_job_count_dict['IT Support']+=1
        elif "it engineer" in job_title:
            my_job_count_dict['IT Support']+=1
        elif "server engineer" in job_title:
            my_job_count_dict['IT Support']+=1
        else:
            my_job_count_dict['Other IT Positions']+=1
#             print (job_title)

def save_job_count_to_csv(my_job_count_dict):
    with open(location, 'wb') as csvfile:
        fieldnames=my_job_count_dict.keys()
        writer=csv.DictWriter(csvfile,fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow(my_job_count_dict)

def plot_the_bar_chart():
    df=pd.read_csv(location)
#     df.head()
    df1 = df.transpose()
    df1.columns = ['number of positions']
    print df1   
    df.T
    df.T.plot.bar()
    plt.title("ANALYSIS of IT JOBS in JOBSDB PORTAL for last %d days"%(input_days))
    plt.xlabel("IT JOB CATEGORIES")
    plt.ylabel("NUMBER OF POSITIONS")
    plt.show()
    
input_days=input("Please enter number of days you want to analyse: ")

while True:
    next_URL_page_to_scrape=url_base+"&page="+str(page_num)
    content =  requests.get(next_URL_page_to_scrape).text
    soup = BeautifulSoup(content, 'html.parser')
    job_title_tag = soup.find_all("a", {"class": "posLink"})
    date_posted_tag = soup.find("meta",  itemprop="datePosted")
    #     pprint (date_posted_tag["content"])
            
    diff=calculate_diff_in_days(date_posted_tag)
#     print ("days difference is %d"%(diff))

    if diff > (input_days-1):
#         print ("more than {}. exit loop".format(input_days))
        break
    
    print ("Iterating IT positions posted on %s in page %d......"%(date_posted_tag["content"],page_num))
            
    sum_each_IT_post_count_in_page(job_title_tag)
                  
    if page_num==total_job_pages:
        break
  
    page_num+=1

# print my_job_count_dict

#save job count to csv file
save_job_count_to_csv(my_job_count_dict)

plot_the_bar_chart()


KeyboardInterrupt: 